# Importing libs

In [2]:
from scipy.fftpack import dct
import librosa
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Creating dataframe

In [4]:
base_path = "TinySOL/"
instruments_type = ["Brass", "Keyboards", "Strings", "Winds"]
instruments = ["Trombone", "Accordion", "Violin", "Flute"]
instruments_df = []


for instrument_type, instrument in zip(instruments_type, instruments):
    full_path = os.path.join(base_path, instrument_type, instrument, "ordinario/")
    note_value = []
    qct_frequencies_per_note = []

    df_instrument = {
        'instrument_name': instrument
    }
    print(full_path)
    
    for file in os.listdir(full_path):
        if file.endswith(".wav"):
            note = file.split('-')[2]
            wav_path = os.path.join(full_path, file)
            y, sr = librosa.load(wav_path, sr=None)
            cqt = librosa.cqt(y, sr=sr, n_bins=84, bins_per_octave=12)

            cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)

            frequencies = librosa.cqt_frequencies(n_bins=84, fmin=librosa.note_to_hz('C2'))
            top_indices = np.argsort(np.mean(cqt_db, axis=1))[::-1][:12]
            top_frequencies = frequencies[top_indices]

            note_value.append(note)
            qct_frequencies_per_note.append(top_frequencies)

    df_instrument['note'] = note_value
    df_instrument['frequencies'] = qct_frequencies_per_note
    df_instrument = pd.DataFrame(df_instrument)
    instruments_df.append(df_instrument)




instruments_df = pd.concat(instruments_df, ignore_index=True)


TinySOL/Brass/Trombone/ordinario/
TinySOL/Keyboards/Accordion/ordinario/
TinySOL/Strings/Violin/ordinario/
TinySOL/Winds/Flute/ordinario/


# Saving dataframe into csv 

In [6]:
instruments_df.head()
instruments_df.to_csv('dataset/dataset.csv', index=False)